# RapidaDB Quickstart

Get started with GPU-accelerated vector search in 5 minutes.

In [ ]:
import torch
from rapidadb import RapidaDB
import time

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

## Create Index and Add Vectors

In [ ]:
# Create index
db = RapidaDB(dim=768, metric='cosine')

# Generate random vectors (in real use: embeddings from your model)
vectors = torch.randn(50000, 768, device='cuda')
db.add(vectors)

print(f"Added {len(db):,} vectors")

## Search for Nearest Neighbors

In [ ]:
# Create queries
queries = torch.randn(100, 768, device='cuda')

# Search
start = time.perf_counter()
distances, indices = db.search(queries, k=10)
elapsed = time.perf_counter() - start

print(f"Search time: {elapsed*1000:.2f} ms")
print(f"Throughput: {len(queries)/elapsed:.0f} QPS")
print(f"Results shape: {indices.shape}")

## Benchmark vs PyTorch

In [ ]:
# RapidaDB
start = time.perf_counter()
dist_rapid, idx_rapid = db.search(queries, k=10)
time_rapid = time.perf_counter() - start

# PyTorch baseline
start = time.perf_counter()
dist_torch = torch.cdist(queries, vectors)
idx_torch = dist_torch.topk(k=10, largest=False).indices
time_torch = time.perf_counter() - start

print(f"RapidaDB: {time_rapid*1000:.2f} ms")
print(f"PyTorch:  {time_torch*1000:.2f} ms")
print(f"Speedup:  {time_torch/time_rapid:.2f}x")